In [ ]:
%load_ext autoreload
%autoreload 2


In [2]:
import sys
sys.path.insert(1, "/home/oru2/project/project")

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import time
# import matplotlib.pyplot as plt
import attacks
from privacy_accountant import PrivacyAccountant
from tqdm import tqdm
from torchvision import datasets, transforms


In [4]:
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")
batch_size = 64

np.random.seed(42)
torch.manual_seed(42)


## Dataloaders
train_dataset = datasets.MNIST('../mnist_data/', train=True, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))
test_dataset = datasets.MNIST('../mnist_data/', train=False, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
attack = attacks.PGD()


In [5]:
import warnings
warnings.simplefilter("ignore")

MAX_GRAD_NORM = 1.2
EPSILON = 1
DELTA = 1e-5
EPOCHS = 20

LR = 1e-3
BATCH_SIZE = 64
MAX_PHYSICAL_BATCH_SIZE = 16

In [6]:
def accuracy(preds, labels):
    return (preds == labels).mean()

In [7]:
from model import fcNet
fc_model = fcNet(784, 128, 10).to(device)
num_epochs = 20
fc_model


fcNet(
  (layers): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): RNNLinear(in_features=784, out_features=128, bias=True)
    (2): ReLU(inplace=True)
    (3): RNNLinear(in_features=128, out_features=128, bias=True)
    (4): ReLU(inplace=True)
    (5): RNNLinear(in_features=128, out_features=10, bias=True)
  )
)

In [67]:
from opacus.validators import ModuleValidator
model = ModuleValidator.fix(fc_model)
ModuleValidator.validate(model, strict=False)

[]

In [8]:
from opacus import PrivacyEngine

privacy_engine = PrivacyEngine()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(fc_model.parameters(), lr=LR)
# model.remove_hooks()
model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    module=fc_model,
    optimizer=optimizer,
    data_loader=train_loader,
    epochs=EPOCHS,
    target_epsilon=EPSILON,
    target_delta=DELTA,
    max_grad_norm=MAX_GRAD_NORM,
    grad_sample_mode = "functorch"
)

print(f"Using sigma={optimizer.noise_multiplier} and C={MAX_GRAD_NORM}")

Using sigma=0.849609375 and C=1.2


In [9]:
from opacus.utils.batch_memory_manager import BatchMemoryManager

def train(model, train_loader, optimizer, epoch, device):
    model.train()
    criterion = nn.CrossEntropyLoss()

    losses = []
    top1_acc = []
    
    for i, (images, target) in enumerate(train_loader):   
        optimizer.zero_grad()
        images = images.to(device)
        target = target.to(device)

        # compute output
        output = model(images)
        loss = criterion(output, target)
        loss.backward()

        optimizer.step()

        preds = np.argmax(output.detach().cpu().numpy(), axis=1)
        labels = target.detach().cpu().numpy()

        # measure accuracy and record loss
        acc = accuracy(preds, labels)

        losses.append(loss.item())
        top1_acc.append(acc)

    epsilon = privacy_engine.get_epsilon(DELTA)
    print(
        f"\tTrain Epoch: {epoch} \t"
        f"Loss: {np.mean(losses):.6f} "
        f"Acc@1: {np.mean(top1_acc) * 100:.6f} "
        f"(ε = {epsilon:.2f}, δ = {DELTA})"
    )

In [10]:
# train(fc_model, train_loader, optimizer, EPOCHS, device)
for epoch in tqdm(range(EPOCHS), desc="Epoch", unit="epoch"):
    train(model, train_loader, optimizer, epoch + 1, device)

Epoch:   0%|          | 0/20 [00:00<?, ?epoch/s]

Epoch:   5%|▌         | 1/20 [00:13<04:25, 13.95s/epoch]

	Train Epoch: 1 	Loss: 2.557529 Acc@1: 11.129321 (ε = 0.26, δ = 1e-05)


Epoch:  10%|█         | 2/20 [00:25<03:42, 12.38s/epoch]

	Train Epoch: 2 	Loss: 2.247010 Acc@1: 23.226968 (ε = 0.34, δ = 1e-05)


...

	Train Epoch: 18 	Loss: 0.704851 Acc@1: 77.268538 (ε = 0.95, δ = 1e-05)


Epoch:  95%|█████████▌| 19/20 [03:28<00:10, 10.93s/epoch]

	Train Epoch: 19 	Loss: 0.672302 Acc@1: 78.420727 (ε = 0.97, δ = 1e-05)


Epoch: 100%|██████████| 20/20 [03:39<00:00, 10.97s/epoch]

	Train Epoch: 20 	Loss: 0.660772 Acc@1: 78.487469 (ε = 1.00, δ = 1e-05)


In [43]:
correct = 0
model.eval()
for j, (images, labels) in enumerate(test_loader):
  images, labels = images.to(device), labels.to(device)
  logits = model(images)
  _, preds = torch.max(logits, 1)
  correct += (preds == labels).sum().item()
  # print('Batch [{}/{}]'.format(j+1, len(test_loader)))
fc_model.train()
print('Accuracy = {}%'.format(float(correct) * 100 / 10000))

Accuracy = 79.7%


In [44]:
from art.attacks.inference.membership_inference import MembershipInferenceBlackBox
from art.estimators.classification import PyTorchClassifier
art_classifier = PyTorchClassifier(
    model=model,
    loss=criterion,
    optimizer=optimizer,
    input_shape=(28, 28),
    nb_classes=10
)

In [15]:
# from art.attacks.evasion.projected_gradient_descent.projected_gradient_descent_pytorch import ProjectedGradientDescentPyTorch

# pgd_attacker = ProjectedGradientDescentPyTorch(art_classifier, eps = 0.1, eps_step=0.01, max_iter=20)

correct = 0
eps = 0.1
model.eval()
for j, (images, labels) in enumerate(test_loader):
  images, labels = images.to(device), labels.to(device)
  adv_images = attack.pgd_untargeted(model, images, labels, 20, eps, 0.01)
  logits = model(images.to(device))
  adv_logits = model(adv_images.to(device))
  _, preds = torch.max(logits, 1)
  _, adv_preds = torch.max(adv_logits, 1)
  correct += (preds == labels).sum().item()
  correct += (adv_preds == labels).sum().item()
  # print('Batch [{}/{}]'.format(j+1, len(test_loader)))
model.train()
print('Accuracy = {}%'.format(float(correct) * 100 / 20000))

tensor(0.5710, device='cuda:0', grad_fn=<NllLossBackward0>) tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        ...,


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ...,

IndexError: pop from empty list

In [13]:
def check_hook_registration(model):
    # print(model.modules)
    for module in model.modules():
        if hasattr(module, 'activations'):
            print(f"Module {module}: activations list length = {module.activations}")

check_hook_registration(model)

Module RNNLinear(in_features=784, out_features=128, bias=True): activations list length = []
Module RNNLinear(in_features=128, out_features=128, bias=True): activations list length = []
Module RNNLinear(in_features=128, out_features=10, bias=True): activations list length = []


In [14]:
from art.attacks.inference.membership_inference import MembershipInferenceBlackBox
from art.estimators.classification import PyTorchClassifier

In [42]:
# optimizer = torch.optim.Adam(fc_model.parameters())
optimizer = optim.RMSprop(fc_model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss()

# Wrap the PyTorch model in ART's PyTorchClassifier
art_classifier = PyTorchClassifier(
    model=model,
    loss=criterion,
    optimizer=optimizer,
    input_shape=(28, 28),
    nb_classes=10
)
attack_train_size = 10000
attack_test_size = 5000

x_train = train_dataset.data
y_train = train_dataset.targets


x_test = test_dataset.data
y_test = test_dataset.targets

attack = MembershipInferenceBlackBox(estimator=art_classifier, attack_model_type="nn")
attack.fit(x_train[:attack_train_size], y_train[:attack_train_size], x_test[:attack_test_size], y_test[:attack_test_size])

mlp_inferred_train_bb = attack.infer(x_train[attack_train_size:], y_train[attack_train_size:])
mlp_inferred_test_bb = attack.infer(x_test[attack_test_size:], y_test[attack_test_size:])

# check accuracy
mlp_train_acc_bb = np.sum(mlp_inferred_train_bb) / len(mlp_inferred_train_bb)
mlp_test_acc_bb = 1 - (np.sum(mlp_inferred_test_bb) / len(mlp_inferred_test_bb))
mlp_acc_bb = (mlp_train_acc_bb * len(mlp_inferred_train_bb) + mlp_test_acc_bb * len(mlp_inferred_test_bb)) / (len(mlp_inferred_train_bb) + len(mlp_inferred_test_bb))


In [43]:
print(f"Members Accuracy: {mlp_train_acc_bb:.4f}")
print(f"Non Members Accuracy {mlp_test_acc_bb:.4f}")
print(f"Attack Accuracy {mlp_acc_bb:.4f}")

Members Accuracy: 0.8110
Non Members Accuracy 0.1310
Attack Accuracy 0.7491


In [102]:
torch.save(fc_model, "models/baseline.pt")